# Feature Engineering for Volatility Prediction

This notebook creates comprehensive features for volatility prediction including:

## Key Features:
- Technical indicators (RSI, MACD, Bollinger Bands, etc.)
- Volatility estimators (Parkinson, Garman-Klass, etc.)
- Regime detection features
- Calendar and seasonal effects
- Cross-asset spillover effects
- Market microstructure indicators
- Indian market specific features (VIX, FII/DII flows)

In [ ]:
import sys
import os

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully")
print(f"Current working directory: {os.getcwd()}")
print(f"Project root: {project_root}")

In [ ]:
# Import project modules
from src.config.settings import get_config
from src.features.basic_features import BasicFeatureEngineer
from src.features.advanced_features import AdvancedFeatureEngineer
from src.features.volatility_estimators import VolatilityEstimators
from src.utils.logging import get_logger

# Initialize configuration and logger
config = get_config()
logger = get_logger(__name__)

print("Project modules imported successfully")

## 1. Load Previously Downloaded Data

Load the data from the previous notebook.

In [ ]:
# Define data directories
raw_data_dir = os.path.join(project_root, 'data', 'raw')
processed_data_dir = os.path.join(project_root, 'data', 'processed')

print(f"Looking for data in:")
print(f"  Raw: {raw_data_dir}")
print(f"  Processed: {processed_data_dir}")

# Check if directories exist
if not os.path.exists(raw_data_dir):
    print(f"⚠ Raw data directory not found. Please run 01_data_download.ipynb first.")
else:
    print(f"✓ Raw data directory found")
    
if not os.path.exists(processed_data_dir):
    print(f"⚠ Processed data directory not found. Please run 01_data_download.ipynb first.")
else:
    print(f"✓ Processed data directory found")

In [ ]:
# Load stock data
stock_data = {}
data_files = [f for f in os.listdir(raw_data_dir) if f.endswith('_stock_data.csv')]

print(f"Found {len(data_files)} stock data files:")

for file in data_files:
    symbol = file.replace('_stock_data.csv', '') + '.NS'
    filepath = os.path.join(raw_data_dir, file)
    
    try:
        data = pd.read_csv(filepath, index_col=0, parse_dates=True)
        stock_data[symbol] = data
        print(f"  ✓ {symbol}: {len(data)} records")
    except Exception as e:
        print(f"  ✗ Failed to load {symbol}: {e}")

print(f"\nLoaded data for {len(stock_data)} symbols")

In [ ]:
# Load VIX data
vix_file = os.path.join(raw_data_dir, 'india_vix_data.csv')
vix_data = None

if os.path.exists(vix_file):
    try:
        vix_data = pd.read_csv(vix_file, index_col=0, parse_dates=True)
        print(f"✓ Loaded VIX data: {len(vix_data)} records")
        print(f"VIX columns: {list(vix_data.columns)}")
    except Exception as e:
        print(f"✗ Failed to load VIX data: {e}")
else:
    print("⚠ VIX data file not found")

# Load volatility estimates
vol_file = os.path.join(processed_data_dir, 'nifty50_volatility_estimates.csv')
volatility_estimates = None

if os.path.exists(vol_file):
    try:
        volatility_estimates = pd.read_csv(vol_file, index_col=0, parse_dates=True)
        print(f"✓ Loaded volatility estimates: {len(volatility_estimates)} records")
        print(f"Volatility columns: {list(volatility_estimates.columns)}")
    except Exception as e:
        print(f"✗ Failed to load volatility estimates: {e}")
else:
    print("⚠ Volatility estimates file not found")

## 2. Basic Feature Engineering

Create basic features from price and volume data.

In [ ]:
# Initialize basic feature engineer
basic_engineer = BasicFeatureEngineer()

# Focus on NIFTY50 for detailed feature engineering
target_symbol = 'NIFTY50.NS'
if target_symbol not in stock_data:
    # Fallback to first available symbol
    target_symbol = list(stock_data.keys())[0]
    print(f"NIFTY50 not available, using {target_symbol} instead")

target_data = stock_data[target_symbol]
print(f"Creating features for {target_symbol}")
print(f"Data shape: {target_data.shape}")
print(f"Date range: {target_data.index[0]} to {target_data.index[-1]}")
print(f"Columns: {list(target_data.columns)}")

In [ ]:
# Create basic features
print("Creating basic features...")

basic_features = basic_engineer.create_features(target_data)

print(f"✓ Basic features created")
print(f"Features shape: {basic_features.shape}")
print(f"Feature columns: {list(basic_features.columns)}")

# Display sample features
display(basic_features.head())
print("\nFeature summary:")
display(basic_features.describe())

In [ ]:
# Visualize basic features
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Plot 1: Returns
if 'returns' in basic_features.columns:
    axes[0, 0].plot(basic_features.index, basic_features['returns'], alpha=0.7)
    axes[0, 0].set_title('Daily Returns')
    axes[0, 0].set_ylabel('Returns')
    axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Log returns
if 'log_returns' in basic_features.columns:
    axes[0, 1].plot(basic_features.index, basic_features['log_returns'], alpha=0.7, color='orange')
    axes[0, 1].set_title('Log Returns')
    axes[0, 1].set_ylabel('Log Returns')
    axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Price momentum
momentum_cols = [col for col in basic_features.columns if 'momentum' in col]
if momentum_cols:
    for col in momentum_cols[:3]:  # Plot first 3 momentum features
        axes[0, 2].plot(basic_features.index, basic_features[col], label=col, alpha=0.8)
    axes[0, 2].set_title('Price Momentum')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)

# Plot 4: Volume features
volume_cols = [col for col in basic_features.columns if 'volume' in col.lower()]
if volume_cols:
    for col in volume_cols[:3]:  # Plot first 3 volume features
        axes[1, 0].plot(basic_features.index, basic_features[col], label=col, alpha=0.8)
    axes[1, 0].set_title('Volume Features')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# Plot 5: High-Low features
hl_cols = [col for col in basic_features.columns if any(x in col.lower() for x in ['high', 'low', 'range'])]
if hl_cols:
    for col in hl_cols[:3]:  # Plot first 3 high-low features
        axes[1, 1].plot(basic_features.index, basic_features[col], label=col, alpha=0.8)
    axes[1, 1].set_title('High-Low Features')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

# Plot 6: Feature correlation heatmap (sample)
sample_features = basic_features.select_dtypes(include=[np.number]).iloc[:, :10]  # First 10 numeric features
if not sample_features.empty:
    corr_matrix = sample_features.corr()
    im = axes[1, 2].imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
    axes[1, 2].set_title('Feature Correlations (Sample)')
    axes[1, 2].set_xticks(range(len(corr_matrix.columns)))
    axes[1, 2].set_yticks(range(len(corr_matrix.columns)))
    axes[1, 2].set_xticklabels(corr_matrix.columns, rotation=45, ha='right', fontsize=8)
    axes[1, 2].set_yticklabels(corr_matrix.columns, fontsize=8)
    plt.colorbar(im, ax=axes[1, 2])

plt.tight_layout()
plt.show()

print("Basic features visualization completed.")

## 3. Advanced Feature Engineering

Create advanced features including technical indicators, regime detection, and calendar effects.

In [ ]:
# Initialize advanced feature engineer
advanced_engineer = AdvancedFeatureEngineer()

print("Creating advanced features...")
print("This may take a while due to complex calculations...")

try:
    # Create advanced features
    advanced_features = advanced_engineer.create_features(target_data)
    
    print(f"✓ Advanced features created")
    print(f"Features shape: {advanced_features.shape}")
    print(f"Feature columns: {list(advanced_features.columns)}")
    
    # Display sample features
    display(advanced_features.head())
    
except Exception as e:
    print(f"⚠ Advanced feature creation failed: {e}")
    print("Continuing with basic features only...")
    advanced_features = pd.DataFrame(index=basic_features.index)

In [ ]:
# Visualize advanced features (if available)
if not advanced_features.empty and len(advanced_features.columns) > 0:
    
    # Select interesting features for visualization
    feature_groups = {
        'Technical': [col for col in advanced_features.columns if any(x in col.lower() for x in ['rsi', 'macd', 'bb'])],
        'Volatility': [col for col in advanced_features.columns if 'vol' in col.lower()],
        'Momentum': [col for col in advanced_features.columns if any(x in col.lower() for x in ['momentum', 'roc'])],
        'Calendar': [col for col in advanced_features.columns if any(x in col.lower() for x in ['month', 'day', 'quarter'])]
    }
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    for i, (group_name, features) in enumerate(feature_groups.items()):
        if i >= 4:  # Only plot first 4 groups
            break
            
        row, col = i // 2, i % 2
        ax = axes[row, col]
        
        if features:
            # Plot up to 3 features from each group
            for feature in features[:3]:
                if feature in advanced_features.columns:
                    data = advanced_features[feature].dropna()
                    if len(data) > 0:
                        ax.plot(data.index, data.values, label=feature, alpha=0.8)
            
            ax.set_title(f'{group_name} Features')
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
        else:
            ax.text(0.5, 0.5, f'No {group_name}\nFeatures Available', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    print("Advanced features visualization completed.")
    
else:
    print("No advanced features available for visualization.")

## 4. Volatility-Specific Features

Create features specifically designed for volatility prediction.

In [ ]:
# Initialize volatility estimators
vol_estimator = VolatilityEstimators()

print("Creating volatility-specific features...")

# Prepare OHLC data
required_cols = ['Open', 'High', 'Low', 'Close']
if all(col in target_data.columns for col in required_cols):
    
    ohlc_data = target_data[required_cols].copy()
    print(f"Using OHLC data: {ohlc_data.shape}")
    
    # Calculate various volatility estimators
    volatility_features = pd.DataFrame(index=ohlc_data.index)
    
    # Simple return volatility (multiple windows)
    returns = ohlc_data['Close'].pct_change()
    for window in [5, 10, 20, 30, 60]:
        vol_simple = vol_estimator.simple_volatility(returns, window=window)
        volatility_features[f'vol_simple_{window}d'] = vol_simple
    
    # High-frequency estimators
    estimators = {
        'parkinson': vol_estimator.parkinson_estimator,
        'garman_klass': vol_estimator.garman_klass_estimator,
        'rogers_satchell': vol_estimator.rogers_satchell_estimator,
        'yang_zhang': vol_estimator.yang_zhang_estimator
    }
    
    for name, estimator_func in estimators.items():
        try:
            if name == 'parkinson':
                vol_est = estimator_func(ohlc_data['High'], ohlc_data['Low'])
            else:
                vol_est = estimator_func(
                    ohlc_data['Open'], ohlc_data['High'], 
                    ohlc_data['Low'], ohlc_data['Close']
                )
            volatility_features[f'vol_{name}'] = vol_est
            print(f"✓ Created {name} volatility estimator")
        except Exception as e:
            print(f"✗ Failed to create {name} estimator: {e}")
    
    # Volatility of volatility
    for col in volatility_features.columns:
        if not volatility_features[col].empty:
            vol_of_vol = volatility_features[col].rolling(20).std()
            volatility_features[f'{col}_vol'] = vol_of_vol
    
    # Volatility ratios and spreads
    vol_cols = [col for col in volatility_features.columns if col.startswith('vol_') and not col.endswith('_vol')]
    
    for i, col1 in enumerate(vol_cols):
        for col2 in vol_cols[i+1:]:
            # Ratio
            ratio = volatility_features[col1] / volatility_features[col2]
            volatility_features[f'{col1}_{col2}_ratio'] = ratio
            
            # Spread
            spread = volatility_features[col1] - volatility_features[col2]
            volatility_features[f'{col1}_{col2}_spread'] = spread
    
    print(f"✓ Created {len(volatility_features.columns)} volatility features")
    
else:
    print("⚠ OHLC data not complete, creating limited volatility features")
    volatility_features = pd.DataFrame(index=target_data.index)
    
    # Simple return volatility only
    if 'Close' in target_data.columns:
        returns = target_data['Close'].pct_change()
        for window in [5, 10, 20, 30]:
            vol_simple = vol_estimator.simple_volatility(returns, window=window)
            volatility_features[f'vol_simple_{window}d'] = vol_simple

print(f"Volatility features shape: {volatility_features.shape}")
if not volatility_features.empty:
    display(volatility_features.head())

## 5. Market-Specific Features

Create features specific to Indian markets including VIX integration and calendar effects.

In [ ]:
# Create market-specific features
market_features = pd.DataFrame(index=target_data.index)

print("Creating market-specific features...")

# VIX-based features
if vix_data is not None and not vix_data.empty:
    print("Adding VIX-based features...")
    
    # Align VIX data with target data
    vix_aligned = vix_data.reindex(target_data.index, method='ffill')
    
    if not vix_aligned.empty:
        vix_col = vix_aligned.columns[0]  # Use first VIX column
        
        # VIX level
        market_features['vix_level'] = vix_aligned[vix_col]
        
        # VIX changes
        market_features['vix_change'] = vix_aligned[vix_col].pct_change()
        market_features['vix_change_5d'] = vix_aligned[vix_col].pct_change(5)
        
        # VIX moving averages
        for window in [5, 10, 20]:
            market_features[f'vix_ma_{window}'] = vix_aligned[vix_col].rolling(window).mean()
            market_features[f'vix_ma_{window}_ratio'] = vix_aligned[vix_col] / market_features[f'vix_ma_{window}']
        
        # VIX percentiles
        for window in [30, 60, 252]:
            market_features[f'vix_percentile_{window}d'] = vix_aligned[vix_col].rolling(window).rank(pct=True)
        
        print(f"✓ Added {sum(1 for col in market_features.columns if 'vix' in col)} VIX features")

# Calendar effects (Indian market specific)
print("Adding calendar effects...")

# Basic calendar features
dates = pd.to_datetime(target_data.index)
market_features['month'] = dates.month
market_features['quarter'] = dates.quarter
market_features['day_of_week'] = dates.dayofweek
market_features['day_of_month'] = dates.day
market_features['week_of_year'] = dates.isocalendar().week

# Month-end and quarter-end effects
market_features['is_month_end'] = (dates + pd.Timedelta(days=1)).month != dates.month
market_features['is_quarter_end'] = (dates + pd.Timedelta(days=1)).quarter != dates.quarter
market_features['days_to_month_end'] = (dates + pd.offsets.MonthEnd(0) - dates).dt.days
market_features['days_to_quarter_end'] = (dates + pd.offsets.QuarterEnd(0) - dates).dt.days

# Indian festival effects (simplified)
# Note: This is a simplified version. In practice, you'd want a comprehensive Indian holiday calendar
diwali_months = [10, 11]  # Diwali typically in Oct/Nov
holi_months = [3, 4]      # Holi typically in Mar/Apr

market_features['is_diwali_season'] = dates.month.isin(diwali_months)
market_features['is_holi_season'] = dates.month.isin(holi_months)

# Monsoon season (affects Indian markets)
monsoon_months = [6, 7, 8, 9]  # June to September
market_features['is_monsoon_season'] = dates.month.isin(monsoon_months)

# Budget season effects
budget_months = [2, 3]  # Budget typically in Feb/Mar
market_features['is_budget_season'] = dates.month.isin(budget_months)

print(f"✓ Added {sum(1 for col in market_features.columns if col not in ['vix_level', 'vix_change', 'vix_change_5d'])} calendar features")

# Cross-asset features (if multiple symbols available)
if len(stock_data) > 1:
    print("Adding cross-asset features...")
    
    # Calculate correlations with other major indices
    other_symbols = [s for s in stock_data.keys() if s != target_symbol]
    
    target_returns = target_data['Close'].pct_change()
    
    for other_symbol in other_symbols[:3]:  # Limit to first 3 for performance
        other_data = stock_data[other_symbol]
        if 'Close' in other_data.columns:
            other_returns = other_data['Close'].pct_change()
            
            # Rolling correlations
            for window in [20, 60]:
                corr = target_returns.rolling(window).corr(other_returns)
                market_features[f'corr_{other_symbol.replace(".NS", "")}_{window}d'] = corr
            
            # Relative performance
            rel_perf = target_returns - other_returns
            market_features[f'rel_perf_{other_symbol.replace(".NS", "")}'] = rel_perf
    
    print(f"✓ Added cross-asset features for {min(3, len(other_symbols))} symbols")

print(f"Market features shape: {market_features.shape}")
if not market_features.empty:
    display(market_features.head())

## 6. Combine All Features

Combine all feature sets and perform final processing.

In [ ]:
# Combine all features
print("Combining all features...")

feature_sets = {
    'basic': basic_features,
    'advanced': advanced_features,
    'volatility': volatility_features,
    'market': market_features
}

# Check which feature sets are available
available_sets = {name: df for name, df in feature_sets.items() if not df.empty}

print(f"Available feature sets: {list(available_sets.keys())}")
for name, df in available_sets.items():
    print(f"  {name}: {df.shape[1]} features")

# Combine features
if available_sets:
    combined_features = pd.concat(list(available_sets.values()), axis=1)
    
    # Remove duplicate columns
    combined_features = combined_features.loc[:, ~combined_features.columns.duplicated()]
    
    print(f"\n✓ Combined features shape: {combined_features.shape}")
    print(f"Total features: {combined_features.shape[1]}")
    
else:
    print("⚠ No features available to combine")
    combined_features = pd.DataFrame()

In [ ]:
# Feature quality assessment
if not combined_features.empty:
    print("=== FEATURE QUALITY ASSESSMENT ===")
    
    # Missing value analysis
    missing_analysis = pd.DataFrame({
        'missing_count': combined_features.isnull().sum(),
        'missing_pct': combined_features.isnull().sum() / len(combined_features) * 100
    }).sort_values('missing_pct', ascending=False)
    
    print(f"Features with >50% missing values: {(missing_analysis['missing_pct'] > 50).sum()}")
    print(f"Features with >25% missing values: {(missing_analysis['missing_pct'] > 25).sum()}")
    print(f"Features with >10% missing values: {(missing_analysis['missing_pct'] > 10).sum()}")
    
    # Show worst missing value features
    if missing_analysis['missing_pct'].max() > 0:
        print("\nTop 10 features with most missing values:")
        display(missing_analysis.head(10))
    
    # Constant features
    numeric_features = combined_features.select_dtypes(include=[np.number])
    constant_features = numeric_features.columns[numeric_features.std() == 0].tolist()
    print(f"\nConstant features (std=0): {len(constant_features)}")
    if constant_features:
        print(f"Constant features: {constant_features[:10]}...")  # Show first 10
    
    # Infinite values
    inf_counts = np.isinf(numeric_features).sum()
    features_with_inf = inf_counts[inf_counts > 0]
    print(f"\nFeatures with infinite values: {len(features_with_inf)}")
    if len(features_with_inf) > 0:
        print(f"Features with inf values: {features_with_inf.head().to_dict()}")
    
    # Summary statistics
    print(f"\nFinal feature summary:")
    print(f"  Total features: {combined_features.shape[1]}")
    print(f"  Numeric features: {len(numeric_features.columns)}")
    print(f"  Date range: {combined_features.index[0]} to {combined_features.index[-1]}")
    print(f"  Total observations: {len(combined_features)}")

In [ ]:
# Clean features for modeling
if not combined_features.empty:
    print("Cleaning features for modeling...")
    
    # Remove features with too many missing values (>50%)
    missing_threshold = 0.5
    missing_pct = combined_features.isnull().sum() / len(combined_features)
    features_to_keep = missing_pct[missing_pct <= missing_threshold].index
    
    cleaned_features = combined_features[features_to_keep].copy()
    print(f"Removed {combined_features.shape[1] - len(features_to_keep)} features with >{missing_threshold*100}% missing values")
    
    # Remove constant features
    numeric_cols = cleaned_features.select_dtypes(include=[np.number]).columns
    constant_cols = cleaned_features[numeric_cols].std() == 0
    non_constant_cols = cleaned_features.columns[~cleaned_features.columns.isin(numeric_cols[constant_cols])]
    
    cleaned_features = cleaned_features[non_constant_cols]
    print(f"Removed {constant_cols.sum()} constant features")
    
    # Replace infinite values with NaN
    numeric_features = cleaned_features.select_dtypes(include=[np.number])
    cleaned_features[numeric_features.columns] = numeric_features.replace([np.inf, -np.inf], np.nan)
    
    # Forward fill missing values (simple strategy)
    cleaned_features = cleaned_features.fillna(method='ffill')
    
    # Backward fill any remaining missing values
    cleaned_features = cleaned_features.fillna(method='bfill')
    
    # Final missing value check
    final_missing = cleaned_features.isnull().sum().sum()
    print(f"Final missing values: {final_missing}")
    
    print(f"\n✓ Cleaned features shape: {cleaned_features.shape}")
    print(f"Final feature count: {cleaned_features.shape[1]}")
    
    # Display final feature summary
    if len(cleaned_features.columns) > 0:
        print("\nFinal features (first 10):")
        display(cleaned_features.iloc[:, :10].head())
        
        print("\nFeature statistics (first 10):")
        display(cleaned_features.iloc[:, :10].describe())
        
else:
    print("⚠ No features available for cleaning")
    cleaned_features = pd.DataFrame()

## 7. Save Engineered Features

Save all engineered features for use in model training.

In [ ]:
# Save engineered features
if not cleaned_features.empty:
    
    # Save cleaned features
    features_file = os.path.join(processed_data_dir, f'{target_symbol.replace(".NS", "")}_engineered_features.csv')
    cleaned_features.to_csv(features_file)
    print(f"✓ Saved engineered features to {os.path.basename(features_file)}")
    
    # Save feature metadata
    feature_metadata = {
        'creation_timestamp': datetime.now().isoformat(),
        'target_symbol': target_symbol,
        'total_features': cleaned_features.shape[1],
        'total_observations': cleaned_features.shape[0],
        'date_range': {
            'start': cleaned_features.index[0].isoformat(),
            'end': cleaned_features.index[-1].isoformat()
        },
        'feature_sets': {
            name: {
                'feature_count': df.shape[1],
                'features': list(df.columns)
            }
            for name, df in available_sets.items()
        },
        'missing_value_threshold': missing_threshold,
        'final_missing_values': final_missing,
        'all_features': list(cleaned_features.columns)
    }
    
    # Save metadata
    import json
    metadata_file = os.path.join(processed_data_dir, f'{target_symbol.replace(".NS", "")}_feature_metadata.json')
    with open(metadata_file, 'w') as f:
        json.dump(feature_metadata, f, indent=2)
    
    print(f"✓ Saved feature metadata to {os.path.basename(metadata_file)}")
    
    # Create feature importance analysis (simple correlation with volatility)
    if 'vol_simple_20d' in cleaned_features.columns:
        target_vol = cleaned_features['vol_simple_20d']
        numeric_features = cleaned_features.select_dtypes(include=[np.number])
        
        feature_correlations = numeric_features.corrwith(target_vol).abs().sort_values(ascending=False)
        
        print("\nTop 10 features correlated with 20-day volatility:")
        display(feature_correlations.head(10))
        
        # Save feature correlations
        corr_file = os.path.join(processed_data_dir, f'{target_symbol.replace(".NS", "")}_feature_correlations.csv')
        feature_correlations.to_csv(corr_file)
        print(f"✓ Saved feature correlations to {os.path.basename(corr_file)}")
    
    print("\n=== FEATURE ENGINEERING COMPLETED ===")
    print(f"Successfully created {cleaned_features.shape[1]} features for {target_symbol}")
    print(f"Data covers {cleaned_features.shape[0]} observations")
    print(f"All files saved to: {processed_data_dir}")
    
else:
    print("⚠ No features to save")